In [1]:
from EduData import get_data
import os


# if not os.path.exists('../../data/skill_builder_data_corrected_collapsed/skill_builder_data_corrected_collapsed.csv'):
#     get_data("2009_skill_builder_data_corrected", "../../data")
# 使用最新版本的assistment2019 :  https://drive.google.com/file/d/1NNXHFRxcArrU0ZJSb9BIL56vmUt5FhlE/view

In [2]:
import pandas as pd
import tqdm

data = pd.read_csv(
    '../../data/2009_skill_builder_data_corrected_collapsed/skill_builder_data_corrected_collapsed.csv',
   usecols=['order_id', 'user_id', 'skill_id', 'problem_id', 'correct','ms_first_response'],
   low_memory=False
).dropna(subset=['order_id', 'user_id', 'skill_id', 'problem_id', 'correct','ms_first_response']).sort_values('order_id')



In [3]:
#data.timeTaken = data.timeTaken.astype(int)
data.head()

,order_id,user_id,problem_id,correct,ms_first_response,skill_id
222489,20224085,73963,76429,0,106016,297
222490,20224095,73963,76430,1,194187,297
222491,20224113,73963,76431,1,12734,297
222492,20224123,73963,76432,1,333484,297
222493,20224142,73963,76433,0,52828,297


In [4]:
data['ms_first_response'] = data['ms_first_response'].apply(int)
data['correct'] = data['correct'].apply(int)

In [5]:
problems = data.problem_id.unique().tolist()
at = data.ms_first_response.unique().tolist()
at= set(int(t/1000) for t in at)

# question id from 1 to #num_skill

problem2id = { p: i+1 for i, p in enumerate(problems) }
at2id = { a: i for i, a in enumerate(at) }

print("number of problems: %d" % len(problems))
print("number of answer time: %d" % len(at))

number of problems: 17751
number of answer time: 1187


In [6]:
skills = data.skill_id.unique().tolist()
#skill2id = { p: i+1 for i, p in enumerate(skills) }
print("number of skills: %d" % len(skills))

# 由于存在复合概念的情况
skill2id = {}
index= 1
for skill in skills:
    s = skill.split('_')
    for _ in s:
        if _ not in skill2id.keys():
            skill2id[_] = index
            index+=1
print("number of single_skills: %d" % len(skill2id.keys()))
for skill in skills:
    if skill not in skill2id.keys():
        skill2id[skill] = index
        index+=1
print("number of total_skills: %d" % len(skill2id.keys()))

number of skills: 149
number of single_skills: 123
number of total_skills: 167


In [7]:
with open('../../data/2009_skill_builder_data_corrected_collapsed/problem2id', 'w', encoding='utf-8') as f:
    f.write(str(problem2id))
with open('../../data/2009_skill_builder_data_corrected_collapsed/skill2id', 'w', encoding='utf-8') as f:
    f.write(str(skill2id))

In [8]:
# import math

# n_difficult=10
# problem2diffcult={}
# for p in problems:
#     temp_data=data[data.problemId==p]
#     answer_num=temp_data.shape[0]
#     if answer_num<20:
#         problem2diffcult[problem2id[p]]=5
#         continue
#     correct_num=temp_data[temp_data.correct==1].shape[0]
#     difficult=correct_num/answer_num*n_difficult
#     difficult=math.floor(difficult)
#     if difficult==10:
#         difficult-=1
#     problem2diffcult[problem2id[p]]=difficult
    


# with open('data/anonymized_full_release_competition_dataset/problem2difficult', 'w', encoding='utf-8') as f:
#     f.write(str(problem2diffcult))
# print(max(problem2diffcult.values()),min(problem2diffcult.values()))

### Notice
As for assistment2009 there not exists information to calculate interval time

In [9]:
import numpy as np
# problems to skills    一个问题多个skill
from collections import defaultdict
import pickle
problem2skill = defaultdict(set)

for s, p in zip(np.array(data.skill_id), np.array(data.problem_id)):
    for _ in s.split('_'):
        problem2skill[problem2id[p]].add(skill2id[_])
with open('../../data/2009_skill_builder_data_corrected_collapsed/problem2skill.pickle', 'wb') as f:
    pickle.dump(problem2skill,f)

In [10]:
def parse_all_seq(students):
    all_sequences = []
    for student_id in tqdm.tqdm(students, 'parse student sequence:\t'):
        student_sequence = parse_student_seq(data[data.user_id == student_id])
        all_sequences.extend([student_sequence])
    return all_sequences


def parse_student_seq(student):
    seq = student
    s = [skill2id[q] for q in seq.skill_id.tolist()]
    a = seq.correct.tolist()
    a = [int(float(i)) for i in a]
    p = [problem2id[p] for p in seq.problem_id.tolist()]
    at = [at2id[int(x/1000)] for x in seq.ms_first_response.tolist()]
    it = [-1]*len(a)
    return s, a, p, it, at              # 都是list



sequences = parse_all_seq(data.user_id.unique())

parse student sequence:	: 100%|██████████| 4163/4163 [00:02<00:00, 1397.52it/s]


In [11]:
# 测试顺序
data[data.user_id == 64525]

,order_id,user_id,problem_id,correct,ms_first_response,skill_id
196010,21441630,64525,54003,0,20797,278
196011,21441970,64525,53991,1,13797,278
196012,21442097,64525,54071,0,14172,278
196013,21442513,64525,54015,1,48813,278
196014,21442851,64525,53987,1,22187,278
...,...,...,...,...,...,...
274115,33273093,64525,59762,0,19048,325
274116,33273154,64525,59763,1,3978,325
274117,33273172,64525,59764,1,14960,325
274118,33273227,64525,59765,1,7363,325


In [12]:
# 测试numpy
ll=[[1,2,3],[1,2],[1,2,3,4]]
#np.array(ll)
np.array(ll,dtype=object)


array([list([1, 2, 3]), list([1, 2]), list([1, 2, 3, 4])], dtype=object)

In [13]:
from sklearn.model_selection import train_test_split, KFold
# 测试train_test_split
train_l,test_l=train_test_split(ll,test_size=0.5)
train_l,test_l

([[1, 2, 3]], [[1, 2], [1, 2, 3, 4]])

In [14]:
# split train data and test data ，random_state参数设置为10，保证分割的结果是可重复的。如果我们多次使用相同的random_state值运行该代码，则每次都会得到相同的分割结果。
train_data, test_data = train_test_split(sequences, test_size=.2, random_state=2023)
# train_data = np.array(train_data)
# test_data = np.array(test_data)
train_data = np.array(train_data,dtype=object)     # numpy版本原因，不然报错
test_data = np.array(test_data,dtype=object)

In [15]:
def sequences2l(sequences, trg_path):
    with open(trg_path, 'a', encoding='utf8') as f:
        for seq in tqdm.tqdm(sequences, 'write data into file: %s' % trg_path):
            s_seq, a_seq, p_seq, it_seq, at_seq = seq
            seq_len = len(s_seq)
            f.write(str(seq_len) + '\n')
            f.write(','.join([str(s) for s in s_seq]) + '\n')
            f.write(','.join([str(a) for a in a_seq]) + '\n')
            f.write(','.join([str(p) for p in p_seq]) + '\n')
            f.write(','.join([str(i) for i in it_seq]) + '\n')
            f.write(','.join([str(a) for a in at_seq]) + '\n')
            
# split into 5 folds
# kfold = KFold(n_splits=5, shuffle=True, random_state=10)      #  random_state=10用于保证交叉验证的结果是可重复的。
# idx = 0
# for train_data_1, valid_data in kfold.split(train_data):
#     sequences2l(train_data[train_data_1], '../../data/2009_skill_builder_data_corrected_collapsed/train' + str(idx) + '.txt')
#     sequences2l(train_data[valid_data], '../../data/2009_skill_builder_data_corrected_collapsed/valid' + str(idx) + '.txt')
#     idx += 1
sequences2l(train_data, '../../data/2009_skill_builder_data_corrected_collapsed/train.txt')
sequences2l(test_data, '../../data/2009_skill_builder_data_corrected_collapsed/test.txt')


write data into file: ../../data/2009_skill_builder_data_corrected_collapsed/train.txt: 100%|██████████| 3330/3330 [00:00<00:00, 12838.30it/s]
write data into file: ../../data/2009_skill_builder_data_corrected_collapsed/test.txt: 100%|██████████| 833/833 [00:00<00:00, 16824.88it/s]


In [16]:
# #测试读取
# ques=[1,2,3,4]
# with open('read_test.txt', 'w', encoding='utf8') as f:
#     f.write(','.join([str(p) for p in ques]) + '\n')
# with open('read_test.txt', 'r', encoding='utf8') as f:
#     for lineID, line in enumerate(f):
#             print(line.split(','))
#             line = line.strip()
#             print(line.split(','))
            
            
            
